<h5> Description of dataset and task

The Entertainment Software Rating Board(ESRB) ratings provide information about the contents of a game so parents and consumers can make informed choices about which game are right for the family (https://www.esrb.org/#rating-categories).

The *Video Games Rating by 'ESRB'* is a dataset composed of **1895** games with 34 of *ESRB Rating Content* along with its name and console exclusivity as features for each game. 
<p> This Notebook aims to create and experiment models capable of classifying games into their proper *ESRB Rating*. </p>


*ESRB Rating Content* (32) refers to the following:
- Alcohol Reference : Reference to and/or images of alcoholic beverages.
- Animated Blood : Discolored and/or unrealistic depictions of blood.
- Blood : Depictions of blood.
- Blood and Gore : 	Depictions of blood or the mutilation of body parts.
- Cartoon Violence : Violent actions involving cartoon-like situations and characters. May include violence where a character is unharmed after the action has been inflicted.
- Crude Humor : Depictions or dialogue involving vulgar antics, including "bathroom" humor.
- Drug Reference : 	Reference to and/or images of illegal drugs.
- Fantasy Violence : Violent actions of a fantasy nature, involving human or non-human characters in situations easily distinguishable from real life.
- Intense Violence : Graphic and realistic-looking depictions of physical conflict. May involve extreme and/or realistic blood, gore, weapons, and depictions of human injury and death.
- Language : Moderate use of profanity.
- Lyrics : References to profanity, sexuality, violence, alcohol, or drug use in music.
- Mature Humor : Depictions or dialogue involving "adult" humor, including sexual references.
- Mild Blood : 	Some blood.
- Mild Cartoon Violence : Some violent actions involving cartoon.
- Mild Fantasy Violence : Some violent actions of a fantasy nature.
- Mild Language : 	Mild to moderate use of profanity.
- Mild Lyrics : Mild References to profanity, sexuality, violence, alcohol, or drug use in music.
- Mild Suggestive Themes : 	some provocative references or materials
- Mild Violence : 	Some scenes involving aggressive conflict.
- No Descriptors : 	No content descriptors
- Nudity : 	Graphic or prolonged depictions of nudity.
- Partial Nudity : Brief and/or mild depictions of nudity.
- Sexual Content : Non-explicit depictions of sexual behavior, possibly including partial nudity.
- Sexual Themes : 	References to sex or sexuality.
- Simulated Gambling : Player can gamble without betting or wagering real cash or currency.
- Strong Language : Explicit and/or frequent use of profanity.
- Strong Sexual Content : 	Explicit and/or frequent depictions of sexual behavior, possibly including nudity.
- Suggestive Themes : 	Provocative references or materials.
- Use of Alcohol :	The consumption of alcoholic beverages.
- Use of Drugs and Alcohol : The consumption of alcoholic and drugs beverages.
- Violence : Scenes involving aggressive conflict. May contain bloodless dismemberment.

*ESRB Rating* refers to the following:
- E : Everyone
- ET : Everyone 10+
- T : teen
- M : Mature

Import revelant libraries.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import torch

from data_loader import DataLoader

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

import optuna
from optuna.trial import TrialState

%matplotlib inline

%load_ext autoreload
%autoreload 2

<h5>  Data Preprocessing

In [ ]:
from jupyterthemes import jtplot
jtplot.style()

In [ ]:
df_esrb = pd.read_csv("Video_Games_esrb_rating.csv")
df_esrb.head(5)

In [ ]:
df_esrb.columns

`strong_janguage` is a clear typo of `strong_language` so we replace that.

In [ ]:
df_esrb = df_esrb.rename(columns={"strong_janguage" : "strong_language"})

Let's display the general `info` of the dataset

In [ ]:
df_esrb.info()

Since theres no `null` values in the dataset, we can proceed

In [ ]:
df_no_desriptors = df_esrb[df_esrb['no_descriptors'] == 1].sum(axis=1) 
df_no_desriptors.shape[0]

Since `no_descriptors` means that none of the _ESRB Content_ is found in the game, we need to check fix games that might contradict that.

In [ ]:
(df_no_desriptors>1).sum()
# print(df_no_desriptors)

In [ ]:
idx = df_no_desriptors[df_no_desriptors>1].index
# df_esrb.iloc[idx].loc['no_descriptors'] = 0

df_esrb.loc[idx,'no_descriptors'] = 0

df_esrb.iloc[idx]['no_descriptors']

Seems like there are 205 entries have the `no_descriptors` flagged wrong.

<h5> Exploratory data analysis

Which _ESRB Rating_ is most prominent?

In [ ]:
ratings = ["E" , "ET" , "T", "M"]

counts = df_esrb['esrb_rating'].value_counts()

plt.bar(ratings, counts)

In [ ]:
plt.pie(counts, labels= ratings, autopct='%.2f')

The _ESRB Rating_ **T** appears to be the most prominent rating.

Next, lets find the most prominent `ESRB Rating Content`

In [ ]:
df_dropped = df_esrb.drop(['title','console','esrb_rating'], axis = 1)
df_count_content = df_dropped.sum(axis=0)
df_count_content.sort_values(axis=0, ascending=False, inplace=True)
df_count_content.plot.bar(x="Content", y="Count", )

We can visibly see that the _ESRB Content_ **blood** is the most prominent while **fantasy_violence** comes close in second.

Let us check the correlelations of each _ESRB Content_

In [ ]:
df_esrb.columns

In [ ]:
correlation = df_dropped.corr()
correlation.style.background_gradient(cmap="coolwarm", axis=None).set_precision(2)

It seems that generally the correlation between each *ESRB Content* is very weak with the strongest correlation of *0.39* between .

<h5> Model Training

Since the dataset already provided a separate test dataset, we wont be needing to split the training dataset. We will only need to create a validation dataset

In [ ]:
label_encoder = preprocessing.LabelEncoder()

In [ ]:
y = label_encoder.fit_transform(df_esrb[['esrb_rating']].values.ravel())
print(y.shape)
print(df_dropped.shape)

In [ ]:
print(y)

In [ ]:
X = df_dropped.values

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, stratify = y, random_state = 42)

In [ ]:
df_test = pd.read_csv("test_esrb.csv")
X_test = df_test.drop(['title','console','esrb_rating'], axis = 1).values
y_test = label_encoder.fit_transform(df_test[['esrb_rating']].values.ravel())
X_test.shape

Convert pandas to tensors

In [ ]:
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)

X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)

X_val = torch.Tensor(X_val)
y_val = torch.Tensor(y_val)

In [ ]:
y_train

In [ ]:
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

<h5> Neural Network

In [ ]:
# 31 -> 4
network = nn.Sequential(
    nn.Linear(31,12),
    nn.ELU(inplace=True),
    nn.Linear(12,6),
    nn.ELU(inplace=True),
    nn.Linear(6,4),
    nn.LogSoftmax(dim = 0)
)
network

In [ ]:
data_loader = DataLoader(X_train,y_train,48)

In [ ]:
loss_fn = nn.CrossEntropyLoss()

![epochs.png](epochs.png)

We trained the model with an epoch of 1500 to see what number of epochs is good for efficiency and accuracy. About 200 is where the "elbow" is seen but we decided around 350 to chase for a slight improvement in accuracy.

In [ ]:
def objective(trial):
    e = 0
    max_epochs = 350
    is_converged = False
    previous_loss = 0
    losses = []

    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop"])
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=False)
    optimizer = getattr(optim, optimizer_name)(network.parameters(), lr=lr)

    while e < max_epochs and is_converged is not True:
    
        current_epoch_loss = 0
        
        X_batch, y_batch = data_loader.get_batch()
        
        # For each batch
        for X, y in zip(X_batch, y_batch):
            X = torch.Tensor(X)
            y = torch.Tensor(y).to(torch.long)
            
            optimizer.zero_grad()
            
            s = network.forward(X)
            
            loss = loss_fn(s,y)
            
            loss.backward()
            
            optimizer.step()
            
            current_epoch_loss += loss.item()
        
        average_loss = current_epoch_loss / len(X_batch)
        losses.append(average_loss)
        
        # Display the average loss per epoch
        # print('Epoch:', e + 1, '\tLoss: {:.6f}'.format(average_loss))
        
        if abs(previous_loss - loss) < 0.0000005:
            is_converged = True
        else:
            previous_loss = loss
            e += 1
    # get accu
    network.eval()

    with torch.no_grad():
        output = network.forward(X_val)
        pred = output.argmax(dim=1, keepdim=True)
        # num_correct = torch.sum(pred == y_val)
    
    accuracy = accuracy_score(pred,y_val)*100.0
    trial.report(accuracy, e)

    return accuracy


        

    

In [ ]:
# study = optuna.create_study(direction = 'maximize')
# study.optimize(objective, n_trials=100)

# print("  Number of finished trials: ", len(study.trials))

# print("Best trial:")
# trial = study.best_trial

# print("  Value: ", trial.value)

# print("  Params: ")
# for key, value in trial.params.items():
#     print("    {}: {}".format(key, value))

Using optuna to optimize the hyperparameters (learning rate and optimizer), it came to a conclusion that Using RMSprop with a learning rate of 0.009992566127730968 resulted in the best validation accuracy, 85.4881266490765%.

In [ ]:
# study.best_trial

<font size="1"> FrozenTrial(number=1, values=[85.4881266490765], datetime_start=datetime.datetime(2022, 6, 30, 20, 25, 36, 105827), datetime_complete=datetime.datetime(2022, 6, 30, 20, 25, 47, 442350), params={'optimizer': 'RMSprop', 'lr': 0.009992566127730968}, distributions={'optimizer': CategoricalDistribution(choices=('Adam', 'RMSprop')), 'lr': UniformDistribution(high=0.01, low=1e-05)}, user_attrs={}, system_attrs={}, intermediate_values={350: 85.4881266490765}, trial_id=1, state=TrialState.COMPLETE, value=None)

In [ ]:
def test_trial(trial):
    e = 0
    max_epochs = 350
    is_converged = False
    previous_loss = 0
    losses = []

    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop"])
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=False)
    optimizer = getattr(optim, optimizer_name)(network.parameters(), lr=lr)

    while e < max_epochs and is_converged is not True:
    
        current_epoch_loss = 0
        
        X_batch, y_batch = data_loader.get_batch()
        
        # For each batch
        for X, y in zip(X_batch, y_batch):
            X = torch.Tensor(X)
            y = torch.Tensor(y).to(torch.long)
            
            optimizer.zero_grad()
            
            s = network.forward(X)
            
            loss = loss_fn(s,y)
            
            loss.backward()
            
            optimizer.step()
            
            current_epoch_loss += loss.item()
        
        average_loss = current_epoch_loss / len(X_batch)
        losses.append(average_loss)
        
        # Display the average loss per epoch
        # print('Epoch:', e + 1, '\tLoss: {:.6f}'.format(average_loss))
        
        if abs(previous_loss - loss) < 0.0000005:
            is_converged = True
        else:
            previous_loss = loss
            e += 1
    # get accu
    network.eval()

    with torch.no_grad():
        output = network.forward(X_test)
        pred = output.argmax(dim=1, keepdim=True)
        # num_correct = torch.sum(pred == y_val)
    
    accuracy = accuracy_score(pred,y_test)*100.0
    trial.report(accuracy, e)

    return accuracy, confusion_matrix(pred,y_test), classification_report(pred,y_test,target_names=label_mapping.keys(),digits=4)

In [ ]:
study.best_trial

In [ ]:
accuracy , confusion_mat, classification_rep = test_trial(study.best_trial)
disp = ConfusionMatrixDisplay(confusion_mat, display_labels = label_mapping)


In [ ]:
disp.plot()
plt.show()
print(classification_rep)
print("The accuracy of the neural network model is : ",accuracy,"%")

The model struggled in predicting some of the "T" rated games as "M". 

<h5> Decision Tree 

In [ ]:

dtc = DecisionTreeClassifier(random_state=42)

In [ ]:
dtc.fit(X_train, y_train)
predictions_train = dtc.predict(X_train)

In [ ]:

def compute_accuracy(predictions, actual):
    # write code here
    return accuracy_score(actual, predictions) * 100

In [ ]:
print("Training accuracy: ", compute_accuracy(y_train, predictions_train),"%")

In [ ]:
predictions = dtc.predict(X_test)
print("Testing accuracy: ", compute_accuracy(y_test, predictions),"%")

<h5> Model Selection and hyperparameter tuning

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
print("Training data label counts:")
print(np.array([unique, counts]))

In [ ]:
ratings = np.array(unique)

In [ ]:
dtc = DecisionTreeClassifier()

In [ ]:
hyperparameters = [
    {
        'criterion': ['gini', 'entropy'],
        'max_depth': [ 5, 10, 20, 30],
        'min_samples_split': [2, 4, 6, 10, 15, 20],
        'max_leaf_nodes': [3, 5, 10, 20, 50, 100],
    }
]

Create  `RandomizedSearchCV` object

In [ ]:
rsc_esrb = RandomizedSearchCV(dtc, n_iter=50, param_distributions=hyperparameters,cv=5,random_state=42)

In [ ]:
rsc_esrb.fit(X_train, y_train)

Get the best parameters found from hyperparameter search

In [ ]:
pd.set_option('display.max_colwidth', None)

rsc_results = pd.DataFrame(rsc_esrb.cv_results_)
rsc_results

In [ ]:
rsc_esrb.best_params_

Get the  best estimator index to in order to get the entry of the best performing model.

In [ ]:
best_index = rsc_esrb.best_index_
best_index

In [ ]:
rsc_results.loc[best_index]

Get the best estimator

In [ ]:
rsc_esrb.best_estimator_

In [ ]:
dtc = rsc_esrb.best_estimator_
dtc.fit(X_train,y_train)

In [ ]:
predictions = dtc.predict(X_test)
print("Test accuracy is : ", compute_accuracy(predictions, y_test), "%")

In [ ]:
print(classification_report(y_test, predictions,target_names=label_mapping.keys(),digits=4))


In [ ]:
cm = confusion_matrix(y_test, predictions, labels=ratings)
display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_mapping.keys())
fig, ax = plt.subplots(figsize=(10,10))
display.plot(ax=ax)

<h5> Insights and conclusion